In [1]:
import random
import time
from copy import copy

import numpy as np
from ipycanvas import Canvas, hold_canvas
from IPython.display import clear_output, display

In [2]:
world_width = 800
world_height = 600
canvas = Canvas(width=world_width, height=world_height)
canvas.fill_style = "black"
canvas.fill_rect(0, 0, world_width, world_height)

In [3]:
class Crawler:
    def __init__(self, x, y, base_color=None):
        self.x = x
        self.y = y

        self.ox = x
        self.oy = y
        self.hunger = 100
        if base_color is None:
            self.r = random.randint(50, 200)
            self.g = random.randint(50, 200)
            self.b = random.randint(50, 200)
        else:
            self.r, self.g, self.b = base_color
        self.color_distance_max = random.randint(50, 170)
        self.color_distance = int(self.color_distance_max / 2)
        self.color_distance_walk_mode = random.choice(["+", "-"])
        self.color_distance_walk_color = random.choice(["r", "g", "b"])
        self.mode = "run"
        self.mode_dict = {}
        self.setupRun()
        """
        if self.mode_dict["steps_direction"] == "y":
            if self.mode_dict["steps_direction_mode"] == "+":
                self.mode_dict["steps_left"] = random.randint(100, 250)"""

    def setupRun(self):
        self.mode = "run"
        self.mode_dict["steps_left"] = random.randint(50, 100)
        self.mode_dict["steps_direction"] = random.choice(["x", "y"])
        self.mode_dict["steps_direction_mode"] = random.choice(["+", "-"])

    def run(self, world):
        updates = {}

        if self.color_distance_walk_mode == "+":
            self.color_distance += 1
        else:
            self.color_distance -= 1

        r = copy(self.r)
        g = copy(self.g)
        b = copy(self.b)
        if self.color_distance_walk_color == "r":
            r += self.color_distance
        elif self.color_distance_walk_color == "g":
            g += self.color_distance
        elif self.color_distance_walk_color == "b":
            b += self.color_distance
        if r > 255:
            r = 255

        if g > 255:
            g = 255

        if b > 255:
            b = 255

        if r < 0:
            r = 0

        if g < 0:
            g = 0

        if b < 0:
            b = 0

        updates[(self.x, self.y)] = f"rgba({r},{g},{b},0.2)"
        valid = False
        while not valid:
            valid = True
            if self.mode == "random":
                direction = random.choice([1, 2, 3, 4])
                if direction == 1:
                    self.y += 1
                elif direction == 2:
                    self.x += 1
                elif direction == 3:
                    self.y -= 1
                elif direction == 4:
                    self.x -= 1
                if random.randint(0, 505) == 5:
                    self.setupRun()
            elif self.mode == "run":
                if self.mode_dict["steps_left"] > 0:
                    if self.mode_dict["steps_direction"] == "x":
                        if self.mode_dict["steps_direction_mode"] == "+":
                            self.x += 1
                        elif self.mode_dict["steps_direction_mode"] == "-":
                            self.x -= 1
                    elif self.mode_dict["steps_direction"] == "y":
                        if self.mode_dict["steps_direction_mode"] == "+":
                            self.y += 1
                        elif self.mode_dict["steps_direction_mode"] == "-":
                            self.y -= 1
                    self.mode_dict["steps_left"] -= 1
                else:
                    self.mode = "random"

            if self.y < 0:
                self.y = world_height

            if self.y > world_height:
                self.y = 0

            if self.x < 0:
                self.x = world_width

        if self.x > world_width:
            self.x = 0

        if self.x <= 0:
            self.x = world_width
        updates[(self.x, self.y)] = f"rgb({r},{g},{b})"
        if self.color_distance_max < self.color_distance or 0 > self.color_distance:
            if self.color_distance_walk_mode == "+":
                self.color_distance_walk_mode = "-"
            else:
                self.color_distance_walk_mode = "+"
            # self.color_distance_walk_color = random.choice(["r", "g", "b"])

        valid = False
        return updates

In [4]:
pallets = []

In [5]:
# hex
html_colors = "#c7522a, #e5c185, #fbf2c4, #74a892, #008585"

In [6]:
html_colors = "#d1dbe4, #a3b7ca, #7593af, #476f95,  #194a7a"

In [7]:
html_colors = "#003f5c,  #58508d,  #bc5090,  #ff6361 ,  #ffa600"

In [8]:
html_colors = "#cac7ff,  #d7cdfe,  #e4d3fc,  #f1dafb ,  #fee0f9"

In [9]:
#
html_colors = " #d9042b,  #730220,  #03658c,  #f29f05,  #f27b50."

In [10]:
html_colors = "#02a5ff,  #ff3300,  #66cc33 ,#ffcc00"

In [11]:
html_colors = "#d6e6ff,  #d7f9f8,  #ffffea,  #fff0d4,  #fbe0e0 ,#e5d4ef"

In [12]:
html_colors = " #2b2dc4,  #3639f7,  #090a2b,  #15165e,#202191,#ffa600"

In [13]:
hexes = [i.strip().strip("#") for i in html_colors.strip().split(",")]
pallets.extend([tuple(int(h[i : i + 2], 16) for i in (0, 2, 4)) for h in hexes])

In [14]:
class World:
    data = {}
    actors = []
    data_since_last_update = {}

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def update(self, update_data):
        self.data.update(update_data)
        self.data_since_last_update.update(update_data)

    def updateCanvas(self, canvas):
        for loc in self.data_since_last_update:
            x, y = loc
            canvas.fill_style = self.data_since_last_update[loc]
            canvas.fill_rect(x, y, 1, 1)
        self.data_since_last_update = {}

In [15]:
world = World(world_width, world_height)
for i in range(100):
    world.actors.append(
        Crawler(
            int(world_width / 2) + random.randint(-10, 10),
            int(world_height / 2) + random.randint(-10, 10),
            base_color=random.choice(pallets),
        )
    )

In [16]:
def update(world):
    for ap in world.actors:
        world.update(ap.run(world))
    world.updateCanvas(canvas)

In [17]:
display(canvas)
update(world)

In [18]:
import ipywidgets as widgets

count = 0
out = widgets.Output(layout={"border": "1px solid white"})
out

In [19]:
while count < 6000:
    with hold_canvas(canvas):
        # Clear the old animation step
        # canvas.clear()
        update(world)
        count += 1
        if count % 100 == 0:
            out.clear_output()
            with out:
                print(count)
    # Animation frequency ~50Hz = 1./50. seconds
    time.sleep(0.001)

In [21]:
canvas.sync_image_data = True
time.sleep(1)
canvas.to_file(f"{time.time()}.png")
canvas.sync_image_data = False